# CSMCC16 Flight Coursework

### Import packages

In [1]:
# import numpy as np
import pandas as pd
import datetime
# import re
from difflib import SequenceMatcher
import PySimpleGUI as sg
import math

### Import data

### Functions for Additional Columns and formats

In [2]:
#calculate datetime values from epoch and arrival time
def calc_arrive_time(df):
    df['Depart_time_datetime'] = pd.to_datetime(df['Depart_time_epoch'],unit='s')
    df['Arrive_epoch'] = pass_data['Depart_time_epoch']+df['Flight_time_min']*60
    df['Arrive_time_datetime'] = pd.to_datetime(df['Arrive_epoch'],unit='s')
    df['Depart_time']=df['Depart_time_datetime'].dt.strftime("%H:%M:%S")


In [3]:
#create route (to use as unique id)
def route(df):
    df['Route'] = df['From_airport_IATA']+'-' + df['Dest_airport_IATA']+'-' + df['Depart_time']

In [4]:
#break route for details
def break_route(df):
    df[['From_airport_IATA','Dest_airport_IATA','Depart_time']]=df['Route'].str.split('-',expand=True)

In [ ]:
#Add new datetime formats
def add_times(df):
    df['Depart_day_month']=df['Depart_time_datetime'].dt.strftime("%a %d/%b/%Y")
    #df['Depart_day_datetime']=pass_data_df['Depart_time_datetime'].dt.strftime("%a %d/%b/%Y %H:%M:%S")
    df['Depart_time']=df['Depart_time_datetime'].dt.strftime("%H:%M:%S")
    df['Arrive_time']=df['Arrive_time_datetime'].dt.strftime("%H:%M:%S")
    df['Depart_day_month']=df['Depart_time_datetime'].dt.strftime("%a %d/%b/%Y")
    #pass_data_df['Depart_day_datetime']=pass_data_df['Depart_time_datetime'].dt.strftime("%a %d/%b/%Y %H:%M:%S")
    df['Depart_time']=df['Depart_time_datetime'].dt.strftime("%H:%M:%S")
    df['Arrive_time']=df['Arrive_time_datetime'].dt.strftime("%H:%M:%S")

In [ ]:
#Add column to allow counting of passengers
def add_1(df):
    df['count']='1'

In [5]:
# Remove any rows that do not fit good_format
def remove_bad_format(df,column,good_format):
    df =df.loc[df[column].str.contains(good_format)==True]
#add reset index?
    return df

In [6]:
# Remove row if not in a given list
def remove_list(df,column,list1):
    df = df[df[column].isin(list1)]
#     df.reset_index(inplace=True)
    return df   

In [29]:
# Calculate number of passengers per flights
def pass_p_flight(df):
    df2 = df[['Flight_id','Passenger_id']]
    df_new = df2.groupby('Flight_id').count()
    df_new.reset_index(inplace=True)
    return df_new 

In [8]:
def remove_if_1(df,column):
    df = df.loc[df[column]!=1]
#     df.reset_index(inplace=True)
    return df

## Setting up the gui output

In [ ]:
sg.SetOptions(
    background_color='#152238',
       text_element_background_color='#152238',
       element_background_color='#152238',
       text_color='#FFFFFF',
       scrollbar_color=None,
       input_elements_background_color='#FFFFFF',
       progress_meter_color = ('green', 'blue')
#        button_color=('white','#475841')
    )

In [9]:
#import csv as dataframe
pass_data = pd.DataFrame(pd.read_csv('AComp_Passenger_data.csv',header=None))
airport_lat_lon = pd.DataFrame(pd.read_csv('Top30_airports_LatLong.csv',header=None))

In [10]:
# Add Column Names
pass_data.columns = ['Passenger_id'
                        ,'Flight_id'
                        ,'From_airport_IATA'
                        ,'Dest_airport_IATA'
                        ,'Depart_time_epoch'
                        ,'Flight_time_min']
airport_lat_lon.columns = ['Airport_name'
                        ,'Airport_IATA_code'
                        ,'Latitude'
                        ,'Longitude']

### Remove missing values

In [11]:
#Remove values where passenger id and flight id are missing
pass_data=pass_data.dropna(subset=['Flight_id','Passenger_id'])

## Error Correction
### Flights

### Flight data only

In [12]:
#Add flight time info 
calc_arrive_time(pass_data)
# add_times(pass_data)

In [13]:
#select subset of data 
flight_info = ['Flight_id'
                      ,'From_airport_IATA'
                      ,'Dest_airport_IATA'
                      ,'Depart_time'         
                        ]
flights = pass_data[flight_info]
#Group data by flight
flights = flights.groupby(flight_info).count()
#reset index
flights.reset_index(inplace=True)
#add route info
route(flights)
# flights

In [14]:
#Remove flights that do not fit the airport format
format_IATA = r"[A-Z][A-Z][A-Z]"
flights=remove_bad_format(flights,'From_airport_IATA',format_IATA)
flights=remove_bad_format(flights,'Dest_airport_IATA',format_IATA)

In [18]:
#Remove flights that do not fit the flight id format
format_flightid = r"[A-Z][A-Z][A-Z][0-9][0-9][0-9][0-9][A-Z]"
flights=remove_bad_format(flights,'Flight_id',format_flightid)

In [19]:
# Remove rows where airport is not in airport list
flights = remove_list(flights
                      ,'From_airport_IATA'
                      ,airport_lat_lon['Airport_IATA_code'
                                      ])
flights = remove_list(flights
                      ,'Dest_airport_IATA'
                      ,airport_lat_lon['Airport_IATA_code'
                                      ])

In [30]:
# Calculate flights per passenger
flights_pass = pass_p_flight(pass_data)
# Remove if just 1 flight
flights_pp = remove_if_1(flights_pass,'Passenger_id')

In [31]:
flights_pp

,Flight_id,Passenger_id
1,ATT7791R,15
2,BER7172M,17
3,DAU2617A,12
4,DKZ3042O,11
5,EWH6301Y,10
6,FYL5866L,20
7,GMO5938W,24
9,HUR0974O,7
10,HZT2506M,14
11,JVY9791G,20


In [32]:
# Final flight list, only keep flights in the flight id list
flights = remove_list(flights
                      ,'Flight_id'
                      ,flights_pp['Flight_id'
                                      ])

In [16]:
flights.reset_index(inplace=True, drop = True)


In [33]:
flights

,Flight_id,From_airport_IATA,Dest_airport_IATA,Depart_time,Route
1,ATT7791R,AMS,DEN,17:13:14,AMS-DEN-17:13:14
2,BER7172M,KUL,LAS,17:26:07,KUL-LAS-17:26:07
3,DAU2617A,CGK,SFO,17:23:06,CGK-SFO-17:23:06
4,DKZ3042O,MIA,SFO,17:05:27,MIA-SFO-17:05:27
5,EWH6301Y,CAN,DFW,17:22:47,CAN-DFW-17:22:47
6,FYL5866L,ATL,HKG,17:25:40,ATL-HKG-17:25:40
7,GMO5938W,LHR,PEK,17:11:57,LHR-PEK-17:11:57
9,HUR0974O,DEN,PVG,17:15:25,DEN-PVG-17:15:25
10,HZT2506M,IAH,AMS,17:12:04,IAH-AMS-17:12:04
11,JVY9791G,PVG,FCO,17:16:01,PVG-FCO-17:16:01


## add to passenger data

In [34]:
#Add route info to pass_data
route(pass_data)

In [51]:
#Select data required
pass_data_slim = pass_data[['Passenger_id','Flight_id','Route']]
pass_data_slim
# flight_id_route = flights[['Flight_id','Route']]

,Passenger_id,Flight_id,Route
0,UES9151GS5,SQU6245R,DEN-FRA-17:14:20
1,UES9151GS5,XXQ4064B,JFK-FRA-17:05:17
2,EZC9678QI6,SOH3431A,ORD-MIA-17:00:49
3,ONL0812DH1,SOH3431A,ORD-MIA-17:00:49
4,CYJ0225CH1,PME8178S,DEN-PEK-17:13:29
5,POP2875LH3,MBA8071P,KUL-PEK-17:04:16
6,WTC9125IE5,MOO1786A,MAD-FRA-16:56:48
7,EDV2089LK5,HUR0974O,DEN-PVG-17:15:25
9,UES9151GS5,GMO5938W,LHR-PEK-17:11:57
10,HCA3158QA6,XXQ4064B,JFK-FRA-17:05:17


In [50]:
# correct flight info
def correct_flight_err (df,column1,column2):
    for ind in df.index:
        if (df[column1][ind] in flights[column1].to_list()) == False:
            print(df[column1][ind])
            #  look up the correct value
            true_column1 = df[column2][ind]
            print(true_column1)
            #  get the row number of correct value
            row = flights[flights[column2] == true_column1].index[0]
            print(row)
            #  take passenger id from passenger data, take other data from correct table
# #             correct_value = 
            df[column1][ind]=flights[column1][row]
            print(df[column1][ind])
        return df
# #             flights.loc[flights.index.intersection('Route',row)]
#          else:
        
#             flights.loc[flights.index.intersection('Route',row)]
#             s.loc[s.index.intersection(labels)].reindex(labels)
#             pass_data.loc['Flight_id',ind]=flights.loc[['Route',row]]
#             print(pass_data.loc['Flight_id',ind])
#             print( pass_data.loc['Flight_id',ind])
#             pass_data.loc['Route',ind]=flights.loc['Route'][row]
#             dfmi.loc[:, ('one', 'second')]
#             pass_data_route.append([pass_data_slim['Passenger_id'][ind]
#                             , flight_id_route['Flight_id'][row]
#                             , flight_id_route['Route'][row]])
#     pass_data_route= pd.DataFrame(pass_data_route, columns=['Passenger_id','Flight_id','Route'])

In [ ]:
# correct flight info
def correct_flight_errors(column1,column2):
    for ind in pass_data.index:
        # if column 1 is correct...
        if (pass_data[column1][ind] in flights[column1].to_list()) == True:
            # take the data from passenger data
            pass_data_route.append([pass_data_slim['Passenger_id'][ind]
                            ,pass_data_slim['Flight_id'][ind]
                            ,pass_data_slim['Route'][ind]])
        # if column 1 is NOT correct...
        else:
            #  look up the correct value
            true_route = (pass_data[column2][ind])
            #  get the row number of correct value
            row = flight_id_route[flight_id_route[column2] == true_route].index[0]
            #  take passenger id from passenger data, take other data from correct table
            pass_data_route.append([pass_data_slim['Passenger_id'][ind]
                            , flight_id_route['Flight_id'][row]
                            , flight_id_route['Route'][row]])
    pass_data_route= pd.DataFrame(pass_data_route, columns=['Passenger_id','Flight_id','Route'])

In [46]:
pass_data

,Passenger_id,Flight_id,From_airport_IATA,Dest_airport_IATA,Depart_time_epoch,Flight_time_min,Depart_time_datetime,Arrive_epoch,Arrive_time_datetime,Depart_time,Route
0,UES9151GS5,SQU6245R,DEN,FRA,1420564460,1049,2015-01-06 17:14:20,1420627400,2015-01-07 10:43:20,17:14:20,DEN-FRA-17:14:20
1,UES9151GS5,XXQ4064B,JFK,FRA,1420563917,802,2015-01-06 17:05:17,1420612037,2015-01-07 06:27:17,17:05:17,JFK-FRA-17:05:17
2,EZC9678QI6,SOH3431A,ORD,MIA,1420563649,250,2015-01-06 17:00:49,1420578649,2015-01-06 21:10:49,17:00:49,ORD-MIA-17:00:49
3,ONL0812DH1,SOH3431A,ORD,MIA,1420563649,250,2015-01-06 17:00:49,1420578649,2015-01-06 21:10:49,17:00:49,ORD-MIA-17:00:49
4,CYJ0225CH1,PME8178S,DEN,PEK,1420564409,1322,2015-01-06 17:13:29,1420643729,2015-01-07 15:15:29,17:13:29,DEN-PEK-17:13:29
5,POP2875LH3,MBA8071P,KUL,PEK,1420563856,572,2015-01-06 17:04:16,1420598176,2015-01-07 02:36:16,17:04:16,KUL-PEK-17:04:16
6,WTC9125IE5,MOO1786A,MAD,FRA,1420563408,184,2015-01-06 16:56:48,1420574448,2015-01-06 20:00:48,16:56:48,MAD-FRA-16:56:48
7,EDV2089LK5,HUR0974O,DEN,PVG,1420564525,1398,2015-01-06 17:15:25,1420648405,2015-01-07 16:33:25,17:15:25,DEN-PVG-17:15:25
9,UES9151GS5,GMO5938W,LHR,PEK,1420564317,1057,2015-01-06 17:11:57,1420627737,2015-01-07 10:48:57,17:11:57,LHR-PEK-17:11:57
10,HCA3158QA6,XXQ4064B,JFK,FRA,1420563917,802,2015-01-06 17:05:17,1420612037,2015-01-07 06:27:17,17:05:17,JFK-FRA-17:05:17


In [25]:
def pass_p_flight(df):
    df2 = df[['Flight_id','Passenger_id']]
    df_new = df2.groupby('Flight_id').count()
#     df_new.reset_index(inplace=True)
    return df_new 

In [59]:
# pass_data_route = correct_flight_err('Flight_id','Route')
# pass_data_route = 
pass_data_slim = correct_flight_err(pass_data_slim,'Route','')
pass_data_slim = correct_flight_err(pass_data_slim,'Flight_id','Route')

In [61]:
# pass_data_slim

In [ ]:
# correct_flight_errors('Flight_id','Route')

In [ ]:
# correct_flight_errors('Route','Flight_id')

In [45]:
pd.set_option("display.max_rows", None, "display.max_columns", None)


In [43]:
# pass_data_route

## Error Correction
### Passenger

In [75]:
#Remove flights that do not fit the airport format
format_pass_id = r"[A-Z][A-Z][A-Z][0-9][0-9][0-9][0-9][A-Z][A-Z][0-9]"
# pass_data_route=
freq_fliers = remove_bad_format(pass_data_slim,'Passenger_id',format_pass_id)
freq_fliers

,Passenger_id,Flight_id,Route
0,UES9151GS5,SQU6245R,DEN-FRA-17:14:20
1,UES9151GS5,XXQ4064B,JFK-FRA-17:05:17
2,EZC9678QI6,SOH3431A,ORD-MIA-17:00:49
3,ONL0812DH1,SOH3431A,ORD-MIA-17:00:49
4,CYJ0225CH1,PME8178S,DEN-PEK-17:13:29
5,POP2875LH3,MBA8071P,KUL-PEK-17:04:16
6,WTC9125IE5,MOO1786A,MAD-FRA-16:56:48
7,EDV2089LK5,HUR0974O,DEN-PVG-17:15:25
9,UES9151GS5,GMO5938W,LHR-PEK-17:11:57
10,HCA3158QA6,XXQ4064B,JFK-FRA-17:05:17


In [71]:
def flight_per_pass(df):
    df2 = df[['Flight_id','Passenger_id']]
    df_new = df2.groupby('Passenger_id').count()
    df_new.reset_index(inplace=True)
    return df_new 

# #  pass_p_flight(freq_fliers)
# # flights per passenger
# # def pass_p_flight(df):
#     df2 = df[['Flight_id','Passenger_id']]
#     df_new = df2.groupby('Passenger_id').count()
#     df_new.reset_index(inplace=True)
#     return df_new 

In [83]:
freq_fliers_pf=flight_per_pass(freq_fliers)
# freq_fliers_pf

In [84]:
#remove poor format
format_pass_id = r"[A-Z][A-Z][A-Z][0-9][0-9][0-9][0-9][A-Z][A-Z][0-9]"
freq_fliers_pf=remove_bad_format(freq_fliers_pf,'Passenger_id',format_pass_id)

In [87]:
freq_fliers_pf = remove_if_1(freq_fliers_pf,'Flight_id')
freq_fliers_pf

,Passenger_id,Flight_id
0,BWI0520BG6,23
1,CDC0302NN5,12
2,CKZ3132BR4,18
3,CXN7304ER2,17
4,CYJ0225CH1,11
5,DAZ3029XA0,23
6,EDV2089LK5,13
7,EZC9678QI6,21
8,HCA3158QA6,21
9,HGO4350KK1,18


In [129]:
############## TURN THIS INTO A FUNCTION ################### 

true = freq_fliers_pf['Passenger_id'].to_list()

for ind in pass_data_slim.index:
    f = pass_data_slim['Passenger_id'][ind]
    if (f in true)==False:
        print(f)
        test_best_true = []
        final_true = []
        for t in true:
                test_best_true.append([t,round(SequenceMatcher(None, t, f).ratio(),3)])
        test_best_true = pd.DataFrame(test_best_true)
    #     print(test_best_true)
        max_true = test_best_true.iloc[test_best_true[1].idxmax(), 0]
#         print(max_true)    
        pass_data_slim['Passenger_id'][ind]=max_true
#         return pass_data_slim

UES9151GS5
UES9151GS5
EZC9678QI6
ONL0812DH1
CYJ0225CH1
POP2875LH3
WTC9125IE5
EDV2089LK5
UES9151GS5
HCA3158QA6
YMH6360YP0
PUD8209OG3
PAJ3974RK1
UES9151GS5
WYU2010YH8
JJM4724RF7
MXU9187YC7
HGO4350KK1
PUD8209OG3
WTC9125IE5
BWI0520BG6
EDV2089LK5
UES9151GS5
JBE2302VO4
UES9151GS5
BWI0520BG6
YMH6360YP0
DAZ3029XA0
PIT2755XC1
WTC9125IE5
POP2875LH3
MXU9187YC7
EZC9678QI6
CYJ0225CH1
DAZ3029XA0
EZC9678QI6
CKZ3132BR4
JBE2302VO4
UES9151GS5
CXN7304ER2
WBE6935NU3
HCA3158QA6
EDV2089LK5
IEG9308EA5
WYU2010YH8
SJD8775RZ4
SJD8775RZ4
HCA3158QA6
CDC0302NN5
BWI0520BG6
HCA3158QA6
KKP5277HZ7
SJD8775RZ4
SPR4484HA6
CKZ3132BR4
DAZ3029XA0
PUD8209OG3
EZC9678QI6
KKP5277HZ7
VZY2993ME1
HCA3158QA6
CYJ0225CH1
JJM4724RF7
UES9151GS5
JJM4724RF7
HCA3158QA6
CYJ0225CH1
SPR4484HA6
WYU2010YH8
JJM4724RF7
CKZ3132BR4
PUD8209OG3
PIT2755XC1
HGO4350KK1
YMH6360YP0
POP2875LH3
SPR4484HA6
DAZ3029XA0
DAZ3029XA0
HGO4350KK1
EZC9678QI6
SJD8775RZ4
DAZ3029XA0
UES9151GS5
LLZ3798PE3
JJM4724RF7
UES9151GS5
UES9151GS5
WBE6935NU3
WBE6935NU3
LLZ3798PE3

In [132]:
pass_data_slim.reset_index(inplace=True, drop = True)
pass_data_slim

,Passenger_id,Flight_id,Route
0,UES9151GS5,SQU6245R,DEN-FRA-17:14:20
1,UES9151GS5,XXQ4064B,JFK-FRA-17:05:17
2,EZC9678QI6,SOH3431A,ORD-MIA-17:00:49
3,ONL0812DH1,SOH3431A,ORD-MIA-17:00:49
4,CYJ0225CH1,PME8178S,DEN-PEK-17:13:29
5,POP2875LH3,MBA8071P,KUL-PEK-17:04:16
6,WTC9125IE5,MOO1786A,MAD-FRA-16:56:48
7,EDV2089LK5,HUR0974O,DEN-PVG-17:15:25
8,UES9151GS5,GMO5938W,LHR-PEK-17:11:57
9,HCA3158QA6,XXQ4064B,JFK-FRA-17:05:17


In [ ]:
#create function to find the best match for a given false value and list for true values
def best_match(f,t):
    global test_best_true
    global final_true
    test_best_true = []
    final_true =[]
    for t in true:
        test_best_true.append([t,round(SequenceMatcher(None, t, f).ratio(),3)])
    test_best_true = pd.DataFrame(test_best_true)
    max_true = test_best_true.iloc[test_best_true[1].idxmax(), 0]
    final_true.append([f,max_true])
    final_true=pd.DataFrame(final_true,columns=['false','true'])
   # print(final_true)

In [ ]:
# best_match(f,true)

In [ ]:
#test each false value
best_match_results = pd.DataFrame()
for f in false:
    best_match(f,true)
    best_match_results=best_match_results.append(final_true)

In [ ]:
best_match_results

## Objective 1: 
### Determine the number of flights from each airport; include a list of any airports not used.

In [ ]:
airport_list = airport_lat_lon['Airport_IATA_code']
airport_list
break_route(pass_data_route)
from_list = pass_data_route['From_airport_IATA']

In [ ]:
#mapper, where key = airport and value = 1 for each airport in from_list
key2val2 = [[a, 1] for a in from_list]
mapper = pd.DataFrame(key2val2, columns=['Airport_IATA_code','value'])
#mapper

#reducer - takes key and val from mapper to provide final key and value
reducer=mapper.groupby(['Airport_IATA_code']).sum()
reducer
reducer.reset_index(inplace=True)

In [ ]:
# merge full list of airports with number of flights from each airport
airport_list=pd.DataFrame(airport_list)
num_flights = airport_list.merge(reducer, how='outer', on=['Airport_IATA_code'])

In [ ]:
num_flights

In [ ]:
# clean up results
num_flights=num_flights.fillna(0)
num_flights=num_flights.sort_values(by=['value','Airport_IATA_code'], ascending=[False,True])
# num_flights.set_index('Airport_IATA_code', inplace=True, drop=True)
# num_flights

## Objective 2
### Create a list of flights based on the Flight id, this output should include the passenger Id, relevant IATA/FAA codes, the departure time, the arrival time (times to be converted to HH:MM:SS format), and the flight times.

## Objective 3
### Calculate the number of passengers on each flight.

In [ ]:
# pass_data_route
# calc_arrive_time(pass_data_route)
#num_flights = airport_list.merge(from_counts_df, how='outer', on=['Airport_IATA_code'])

In [ ]:
flights = pass_data_route['Flight_id']

In [ ]:
#mapper, where key = airport and value = 1 for each airport in from_list
key2val2 = [[a, 1] for a in flights]
mapper = pd.DataFrame(key2val2, columns=['Flight_id','value'])
#mapper

#reducer - takes key and val from mapper to provide final key and value
num_passengers=mapper.groupby(['Flight_id']).sum()
#num_passengers
#reducer
num_passengers.reset_index(inplace=True)

In [ ]:
# clean up results
#num_passengers=num_flights.fillna(0)
num_passengers=num_passengers.sort_values(by=['value','Flight_id'], ascending=[False,True])
num_passengers.set_index('Flight_id', inplace=True, drop=True)
num_passengers

## Objective 4
### Calculate the line-of-sight (nautical) miles for each flight and the total travelled by each passenger and thus output the passenger having earned the highest air miles.

In [ ]:
## NEED TO REDO!!

In [ ]:
# pass_data_route (Passenger_id,Flight_id,Route,From_airport_IATA,Dest_airport_IATA,Depart_time)
# airport_lat_lon (Airport_IATA_code,Latitude,Longitude)

In [ ]:
def calculate_miles(df_in):
    global miles_per_flight
    miles_per_flight = []
    for ind in df_in.index:
            #Take from airport
            p= df_in['Passenger_id'][ind]
            x= df_in['From_airport_IATA'][ind]
            #Find it in airport lat long list
            row = (airport_lat_lon[airport_lat_lon['Airport_IATA_code'] == x].index[0])
            #Give me the latitude and longitude
            a = airport_lat_lon['Latitude'][row]
            b = airport_lat_lon['Longitude'][row]
            #Take dest airport
            y = df_in['Dest_airport_IATA'][ind]
            #Find it in airport lat long list
            row = (airport_lat_lon[airport_lat_lon['Airport_IATA_code'] == y].index[0])
            #Give me the latitude and longitude
            c = airport_lat_lon['Latitude'][row]
            d = airport_lat_lon['Longitude'][row]
            miles = math.sqrt((c-a)**2+(d-b)**2)
            miles_per_flight.append([df_in['Passenger_id'][ind],miles])
    miles_per_flight= pd.DataFrame(miles_per_flight
                                   , columns=['Passenger_id','Miles']
                                  )    


In [ ]:
calculate_miles(pass_data_route)

In [ ]:
miles_per_flight
# pd.DataFrame(miles_per_flight)
# pass_data_route= pd.DataFrame(pass_data_route, columns=['Passenger_id','Flight_id','Route'

In [ ]:
key2val2 = pd.DataFrame()
for index,row in miles_per_flight.iterrows():
    key2val2=key2val2.append(miles_per_flight.loc[index])
    mapper = pd.DataFrame(key2val2, columns=['Miles','Passenger_id'])
mapper

In [ ]:
#reducer - takes key and val from mapper to provide final key and value
miles_per_pass=mapper.groupby(['Passenger_id']).sum().round(1)
#reducer
miles_per_pass.reset_index(inplace=True)

In [ ]:
# clean up results
#num_passengers=num_flights.fillna(0)
miles_per_pass=miles_per_pass.sort_values(by=['Miles'], ascending=[False])
miles_per_pass.set_index('Passenger_id', inplace=True, drop=True)
miles_per_pass